
# CNN Didática (PyTorch) — Classificação do Digits (8×8)

Este notebook implementa uma **Rede Neural Convolucional (CNN)** em **PyTorch** usando o **dataset Digits** do `scikit-learn` (imagens 8×8, sem necessidade de internet).  
O objetivo é mostrar, de forma **autocontida e prática**:

- Como preparar os dados (tensores 1×8×8);  
- Como definir e treinar uma **CNN simples**;  
- Como avaliar (acurácia e **matriz de confusão**);  
- Como **visualizar filtros** aprendidos e um **feature map** intermediário.

> Se você já viu CNNs com MNIST/CIFAR, aqui a lógica é idêntica — apenas mudamos para o `digits` por ser **offline**.


## 1) Imports

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.mps.is_available() else "cpu")
device



## 2) Dados — `sklearn.datasets.load_digits()`

- Imagens **8×8** em escala de cinza (valores de 0 a 16).  
- Convertidas para **float32** e normalizadas para `[0,1]`.  
- Reformatadas para o formato **(N, 1, 8, 8)** esperado pela CNN.


In [ ]:
digits = load_digits()
X = digits.images  # shape (N, 8, 8), valores de 0..16
y = digits.target  # rótulos de 0..9

X = (X / 16.0).astype(np.float32)
X = X[:, None, :, :]

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=SEED, stratify=y
)

X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.long)
X_test_t  = torch.tensor(X_test,  dtype=torch.float32)
y_test_t  = torch.tensor(y_test,  dtype=torch.long)

train_ds = TensorDataset(X_train_t, y_train_t)
test_ds  = TensorDataset(X_test_t,  y_test_t)

train_loader = DataLoader(train_ds, batch_size=64, shuffle=True)
test_loader  = DataLoader(test_ds,  batch_size=256, shuffle=False)

X_train_t.shape, y_train_t.shape, X_test_t.shape, y_test_t.shape



## 3) Modelo — CNN simples

Arquitetura didática para 8×8:

- `Conv2d(1 → 16, kernel=3, padding=1)` + ReLU  
- `MaxPool2d(2)` → reduz 8×8 → 4×4  
- `Conv2d(16 → 32, kernel=3, padding=1)` + ReLU  
- `MaxPool2d(2)` → reduz 4×4 → 2×2  
- `Flatten` → `Linear(32×2×2 → 64)` → ReLU → `Linear(64 → 10)`


In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(32*2*2, 64),
            nn.ReLU(),
            nn.Linear(64, 10)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = SimpleCNN().to(device)
model



## 4) Treino

Usaremos **CrossEntropyLoss** e **Adam**.  
O loop reporta `loss` de treino por época e calcula a **acurácia** no conjunto de teste.


In [ ]:
def train(model, train_loader, test_loader, epochs=10, lr=1e-3):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    history = {"train_loss": [], "test_acc": []}

    for epoch in range(1, epochs+1):
        model.train()
        running = 0.0
        for xb, yb in train_loader:
            xb = xb.to(device)
            yb = yb.to(device)
            logits = model(xb)
            loss = criterion(logits, yb)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running += loss.item() * xb.size(0)

        epoch_loss = running / len(train_loader.dataset)

        model.eval()
        all_preds = []
        all_true = []
        with torch.no_grad():
            for xb, yb in test_loader:
                xb = xb.to(device)
                logits = model(xb)
                preds = torch.argmax(logits, dim=1).cpu().numpy()
                all_preds.append(preds)
                all_true.append(yb.numpy())
        import numpy as np
        all_preds = np.concatenate(all_preds)
        all_true  = np.concatenate(all_true)
        from sklearn.metrics import accuracy_score
        acc = accuracy_score(all_true, all_preds)

        history["train_loss"].append(epoch_loss)
        history["test_acc"].append(acc)

        print(f"Época {epoch:02d} | Loss treino: {epoch_loss:.4f} | Acurácia teste: {acc:.4f}")
    return history

hist = train(model, train_loader, test_loader, epochs=12, lr=1e-3)



## 5) Curvas (Loss de treino e Acurácia de teste)

**Uma figura por gráfico** (sem subplots).


In [ ]:
plt.figure(figsize=(7,4))
plt.plot(hist["train_loss"])
plt.xlabel("Época")
plt.ylabel("Loss de treino")
plt.title("Curva de Loss (treino)")
plt.grid(alpha=0.3)
plt.show()

plt.figure(figsize=(7,4))
plt.plot(hist["test_acc"])
plt.xlabel("Época")
plt.ylabel("Acurácia (teste)")
plt.title("Curva de Acurácia (teste)")
plt.grid(alpha=0.3)
plt.show()



## 6) Avaliação final — Matriz de confusão e relatório


In [ ]:
model.eval()
all_preds = []
all_true = []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        logits = model(xb)
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        all_preds.append(preds)
        all_true.append(yb.numpy())
import numpy as np
all_preds = np.concatenate(all_preds)
all_true  = np.concatenate(all_true)

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
acc = accuracy_score(all_true, all_preds)
cm  = confusion_matrix(all_true, all_preds)
print(f"Acurácia (teste): {acc:.4f}")
print("Matriz de confusão:\n", cm)
print("\nClassification report:\n", classification_report(all_true, all_preds))



## 7) Visualizando filtros (1ª camada conv)


In [ ]:
with torch.no_grad():
    conv1_weights = model.features[0].weight.cpu().numpy()

print("conv1_weights shape:", conv1_weights.shape)

for i in range(conv1_weights.shape[0]):
    w = conv1_weights[i, 0, :, :]
    plt.figure(figsize=(3,3))
    plt.imshow(w, cmap="gray")
    plt.title(f"Filtro conv1 #{i}")
    plt.axis("off")
    plt.show()



## 8) Visualizando um feature map (após 1ª conv + ReLU)


In [ ]:
idx = 0
sample_img = X_test_t[idx:idx+1].to(device)

with torch.no_grad():
    conv1 = model.features[0]
    relu  = model.features[1]
    fmap  = relu(conv1(sample_img))

fmap_np = fmap[0].cpu().numpy()

plt.figure(figsize=(3,3))
plt.imshow(sample_img[0,0].cpu(), cmap="gray")
plt.title("Imagem original (8×8)")
plt.axis("off")
plt.show()

plt.figure(figsize=(3,3))
plt.imshow(fmap_np[0], cmap="gray")
plt.title("Feature map (canal 0)")
plt.axis("off")
plt.show()



## 9) Conclusão

- A CNN aprendeu **filtros locais** (3×3) úteis para distinguir dígitos.  
- Vimos o ciclo completo: **dados → CNN → treino → avaliação → visualização**.  
- A mesma lógica escala para imagens maiores (28×28, 32×32) com arquiteturas um pouco mais profundas.
